In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import collections
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import precision_score, recall_score, f1_score
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from scipy import sparse
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB

In [0]:
df = pd.read_csv("/content/drive/My Drive/TCS/Train.csv")


In [0]:
copy_df=df.copy()

In [0]:
copy_df.dropna(inplace=True)  

In [0]:
copy_df.drop(columns=["Id"],inplace=True)

In [0]:
copy_df.reset_index(inplace=True)
copy_df.drop(columns=["index"], inplace=True)

In [0]:
def cleaning(text):
  list = []
  codes = []
  for i in range(copy_df.shape[0]):
    clean = re.compile('<.*?>')
    sentence = re.sub(clean, '',  copy_df[text][i] )
    sentence = re.sub(r'C\+\+', 'CPP', sentence)
    sentence = re.sub(r'C#', 'CSHARP', sentence)
    sentence = re.sub(r'[^a-zA-Z0-9]+', ' ', sentence)
    sentence = re.sub(r"(?:^| )+[a-bd-qs-zA-BD-QS-Z](?=$| )+", "", sentence)
    sentence =" ".join(sentence.split())
    sentence = sentence.lower()
    list.append(sentence)
    if text=="Body":
      code = str( re.findall(r"<code>(.*?)</code>", df["Body"][i], flags=re.DOTALL) ) or ""
      sentence = re.sub(r"<code>(.*?)</code>", " ", df["Body"][i], flags=re.MULTILINE|re.DOTALL)
      codes.append(code)
  copy_df["code"]=codes
  copy_df[text] = list


In [0]:
cleaning("Title")
cleaning("Body")

In [0]:
combined_df = pd.DataFrame()
combined_df["Combined"] = copy_df["Title"] + " " + copy_df["Body"] + " " + copy_df["Code"]

In [13]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
for i in range(combined_df.shape[0]):
  words = word_tokenize(copy_df.iloc[i,0])
  t = " ".join(str(stemmer.stem(word)) for word in words if word not in stop_words  )
  tok_stop_stem_list.append(t)

In [0]:
vectorizer = CountVectorizer(ngram_range=(1,1), tokenizer=lambda x: x.split(), min_df=0.00009, max_features=25000)
data1 = vectorizer.fit_transform(combined_df)
ickle.dump(vectorizer.vocabulary_, open("final/vector1.pickle", "wb"))

In [0]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
data2 = vectorizer.fit_transform(copy_df['Tags'])
pickle.dump(vectorizer.vocabulary_, open("final/vector2.pickle", "wb"))

In [0]:
data1_train, data1_test, data2_train, data2_test = train_test_split(data1, data2, test_size=0.05, random_state=101)

In [0]:
classfier_1 = OneVsRestClassifier(LogisticRegression(C=0.1, penalty='l2', verbose=1))
classifier_1.fit(data1_train,data2_train)
pickle.dump(classfier1, open("classifier/clf_1.pickle", "wb"))

In [0]:
classifier_2 = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=1, penalty='l2',n_jobs=-1))
classifier_2.fit(data1_train,data2_train)
pickle.dump(clf_sgd_1, open("classifier/clf_2.pickle", "wb"))

In [0]:
classifier_3 = OneVsRestClassifier(SVC())
classifier_3.fit(X_train_1_1, y_train)
pickle.dump(clf_svc, open("classifier/clf_3.pickle", "wb"))

In [0]:
prediction = -1
for i in range(1,4):
  model = pickle.load(open("classifier/clf_"+i+".pickle", "rb"))
  max = model.predict(data1_test)
  if prediction > max:
    prediction = max

In [0]:
tags_vector = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("final/vector2.pickle", "rb")))
tags_vector._validate_vocabulary()
ques_vector = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("final/vector1.pickle", "rb")))
ques_vector._validate_vocabulary()

In [0]:
Results = pd.DataFrame({"Questions": ques_vector.inverse_transform(data1_test),
                        "Tags": tags_vector.inverse_transform(prediction)})

In [0]:
Results.to_csv("Result/Results.csv", index=True)